In [1]:
#************************************************************************************
# Rezwan Matin
# Thesis B
# Filename: ML_RAVDESS_TESS_2.py
# Date: 12/11/19
#
# Objective:
# Testing RAVDESS+TESS data with background noise.
#
#*************************************************************************************

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as rosa
import glob
import os
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix

In [2]:
# Save directory path in 'path'
path = r'C:\Books\Texas State Books\Fall 2019\Thesis A\Corpus\Simulated\RAVDESS+TESS\All_Background Noise'

# Declare a dummy Numpy array (row vector)
result_array = np.empty([1,54])

# Create a list of audio file names 'file_list'
file_list = os.listdir(path)

i=0

for filename in file_list:
    
    # Read WAV file. 'rosa.core.load' returns sampling frequency in 'fs' and audio signal in 'sig'
    sig, fs = rosa.core.load(path + '\\' + file_list[i], sr=None)
    
    # 'rosa.feature.mfcc' extracts n_mfccs from signal and stores it into 'mfcc_feat'
    mfcc_feat = rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26)
    
    # Calculate the average mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    avg_mfcc_feat = np.mean(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the standard deviation of mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.std' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    std_mfcc_feat = np.std(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the average zero crossing rate (utterance-level feature) using 'rosa.feat.zero_crossing_rate()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    zcross_feat = rosa.feature.zero_crossing_rate(sig)
    avg_zcross_feat = np.mean(rosa.feature.zero_crossing_rate(y=sig).T,axis=0)
    
    # Append the three 1D arrays into a single 1D array called 'feat'.
    feat0 = np.append(avg_mfcc_feat, std_mfcc_feat, axis=0)
    
    feat1 = np.append(feat0, avg_zcross_feat, axis=0)
    
    # Save emotion label from file name. 'path' contains directory's address, 'file_list' contains file name, and '\\' joins the two to form file's address
    label = os.path.splitext(os.path.basename(path + '\\' + file_list[i]))[0].split('-')[2]
    
    # Create a new Numpy array 'sample' to store features along with label
    sample = np.insert(feat1, obj=53, values=label)
    
    result_array = np.append(result_array, sample)
    
    i+=1

# Print out the 1D Numpy array
result_array

array([8.35110782e-316, 3.27998720e-317, 5.30355226e+180, ...,
       6.67029903e+000, 1.14096902e-001, 8.00000000e+000])

In [3]:
result_array.shape

(229014,)

In [4]:
# Convert 1D Numpy array to 2D array
result_array = np.reshape(result_array, (i+1,-1))

# Delete first dummy row from 2D array
result_array = np.delete(result_array, 0, 0)

# Print final 2D Numpy array 
result_array.shape

(4240, 54)

In [5]:
df = pd.DataFrame(data=result_array)
# Label only the last (target) column
df = df.rename({53: "Emotion"}, axis='columns')
# Delete unnecessary emotion data (calm)
df.drop(df[df['Emotion'] == 2.0].index, inplace = True)
df['Emotion'].replace({1.0: "Neutral", 3.0: "Happy", 4.0: "Sad", 5.0: "Angry", 6.0: "Fearful", 7.0: "Disgust", 8.0: "Surprised"}, inplace=True)
# Reset row (sample) indexing
df = df.reset_index(drop=True)
df.head(12)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,Emotion
0,-370.811277,200.945609,-25.601894,12.781816,-23.361012,12.480950,-7.339299,-13.182599,-9.380556,-4.284814,...,5.384720,4.222119,4.566760,4.612926,4.793089,4.647000,5.119929,4.119411,0.034162,Neutral
1,-366.069746,199.358091,-26.372811,12.007660,-23.430119,12.148651,-8.856314,-11.990725,-10.876269,-3.940869,...,5.253634,4.672116,4.409851,5.220854,4.638651,4.641891,4.555992,4.670715,0.035070,Neutral
2,-367.049981,200.153470,-26.128286,13.793804,-24.219043,12.738294,-8.241901,-13.965553,-9.749571,-4.039100,...,5.157424,4.234278,4.268888,4.813564,4.822563,5.016220,4.948770,4.537225,0.035434,Neutral
3,-369.243934,200.255975,-26.802738,13.414320,-23.770417,11.575530,-7.762641,-12.993787,-9.703835,-4.745808,...,4.794925,4.410998,4.548908,4.777634,4.585788,4.987556,4.711527,4.260513,0.034558,Neutral
4,-373.014555,202.287711,-25.378644,12.692684,-24.181430,12.424396,-6.634761,-14.236648,-8.780441,-4.394664,...,5.090342,4.399502,4.426152,4.788078,4.372947,4.622673,4.822164,4.126837,0.033748,Neutral
5,-372.649868,201.983061,-26.475035,12.527525,-24.256549,12.185351,-6.724715,-14.481719,-8.248213,-5.088603,...,4.936861,4.176942,4.225872,4.637436,4.512505,4.746493,4.848282,4.283253,0.034124,Neutral
6,-367.799723,198.911957,-24.865549,12.917156,-24.776742,13.426679,-8.249437,-13.896153,-9.868086,-3.332101,...,5.238720,4.307449,4.773085,4.758333,4.831739,4.681074,4.651671,4.126506,0.035946,Neutral
7,-369.898067,200.121781,-25.290541,13.452877,-24.315592,13.196527,-8.291886,-12.239270,-9.568036,-4.879833,...,5.483584,4.493953,4.416664,4.776289,5.124567,4.787531,4.834017,4.393637,0.035156,Neutral
8,-373.579880,203.821938,-24.900205,14.049376,-22.587316,13.227645,-4.889046,-13.473050,-7.067546,-3.792451,...,5.197421,4.348918,4.320763,4.948473,4.637800,4.713746,4.679600,4.141800,0.032786,Neutral
9,-353.494818,190.893053,-19.030947,11.677804,-23.301991,14.005309,-10.621754,-10.408835,-12.782006,-3.391942,...,5.317658,4.836061,4.866135,5.410886,5.225579,5.387019,6.027842,5.473536,0.035009,Neutral


In [6]:
df['Emotion'].value_counts()

Surprised    592
Sad          592
Angry        592
Fearful      592
Disgust      592
Happy        592
Neutral      496
Name: Emotion, dtype: int64

In [7]:
# Balance the dataset for equal number of samples for each class.
# Separate majority and minority classes
df_minority = df[df.Emotion=="Neutral"]
df_majority3 = df[df.Emotion=="Happy"]
df_majority4 = df[df.Emotion=="Sad"]
df_majority5 = df[df.Emotion=="Angry"]
df_majority6 = df[df.Emotion=="Fearful"]
df_majority7 = df[df.Emotion=="Disgust"]
df_majority8 = df[df.Emotion=="Surprised"]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=592,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority3, df_majority4, df_majority5, df_majority6, df_majority7, df_majority8])
 
# Display new class counts
df_upsampled.Emotion.value_counts()

# Reset row (sample) indexing
df_upsampled = df_upsampled.reset_index(drop=True)

df_upsampled['Emotion'].value_counts()

Neutral      592
Surprised    592
Sad          592
Angry        592
Fearful      592
Disgust      592
Happy        592
Name: Emotion, dtype: int64

In [8]:
# Extract target feature 'Emotion' in a vector y. Indexing from 0
y = df_upsampled.iloc[0:4144, 53].values
# Extract features 'buying' and 'safety' in a vector X. Indexing from 0
X = df_upsampled.iloc[0:4144, list(range(53))].values
print(y)

['Neutral' 'Neutral' 'Neutral' ... 'Surprised' 'Surprised' 'Surprised']


In [9]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

# Create an object 'sc'
sc = StandardScaler()

# Scale training data
sc.fit(X_train)
X_train_std = sc.transform(X_train)

In [10]:
# Create SVM model with hyperparameters
#svm = SVC(kernel='sigmoid', C=10.0, random_state=1)

# Best settings from GridSearchCV
svm = SVC(kernel='rbf', C=6.78, gamma=0.0588, random_state=1)

# This is training the model
svm.fit(X_train_std, y_train)

SVC(C=6.78, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0588, kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [11]:
# Scale test data
sc.fit(X_test)
X_test_std = sc.transform(X_test)

# Test the model data
y_pred = svm.predict(X_test_std)

In [12]:
# Print out the performance metrics
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Training Accuracy: %.2f' % svm.score(X_train_std, y_train))
#print('Test Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Test Accuracy: %.2f' % svm.score(X_test_std, y_test))

# Print out the performance metrics in a text file.
#text_file = open("Emotion Classifier Performance Metrics.txt", "w")
#text_file.write('Misclassified samples: %d' % (y_test != y_pred).sum())
#text_file.write('Training Accuracy: %.2f' % svm.score(X_train_std, y_train))
#text_file.write('Test Accuracy: %.2f' % svm.score(X_test_std, y_test))
#text_file.close()

Misclassified samples: 122
Training Accuracy: 0.96
Test Accuracy: 0.88
